In [3]:
!pip3 install pandas
!pip3 install requests
!pip3 install streamlit
!pip3 install streamlit_tags
!pip3 install sentence_transformers
!pip3 install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.3 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 346 kB 38.5 MB/s eta 0:00:01
     |████████████████████████████████| 5.3 MB 69.4 MB/s eta 0:00:01
     |████████████████████████████████| 507 kB 40.0 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.7 MB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 731 kB 59.9 MB/s eta 0:00:01
     |████████████████████████████████| 30.7 MB 40.1 MB/s eta 0:00:01
     |█████████████████

In [2]:
# Import necessary packages.
import requests
import pandas as pd
import os

/Users/huimin.yang/huimin_projects/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
file_path = "data.csv"
df = pd.read_csv(file_path)
df.head()

,program_key,primary_school,difficulty_level,duration,program_title,generic_program_title,part_title,generic_part_title,part_summary,lesson_title,generic_lesson_title,lesson_summary,locale
0,nd083,Business,Beginner,1 Months,Digital Freelancer,Digital Freelancer,Managing Freelancing Projects,Managing Freelancing Projects,This course will prepare you for managing free...,Creating a Process to Source Clients,Creating a Process to Source Clients,This lesson will help you source clients for y...,en-us
1,nd064,Cloud Computing,Intermediate,4 Months,Cloud Native Application Architecture,Cloud Native Application Architecture,Cloud Native Fundamentals,Cloud Native Fundamentals,"Throughout this course, students will learn ho...",Container Orchestration with Kubernetes,Container Orchestration with Kubernetes,"In this lesson, you will learn how to use Dock...",en-us
2,cs271,Artificial Intelligence,Intermediate,4 Months,Intro to Artificial Intelligence,Intro to Artificial Intelligence,Intro to Artificial Intelligence,Intro to Artificial Intelligence,NaN,Problem Set 4,Problem Set 4,NaN,en-us
3,nd320,Artificial Intelligence,Advanced,4 Months,AI for Healthcare,AI for Healthcare,Applying AI to Wearable Device Data,Applying AI to Wearable Device Data,NaN,Intro to Digital Sampling & Signal Processing,Intro to Digital Sampling & Signal Processing,"A brief tour through sampling theory, signal p...",en-us
4,cs291,Programming and Development,NaN,2 Months,Interactive 3D Graphics,Interactive 3D Graphics,Interactive 3D Graphics,Interactive 3D Graphics,NaN,Colors and Materials,Colors and Materials,NaN,en-us


In [3]:
# build lesson-level combined text
def combine_lesson_text(row):
    fields = [
        f"Lesson Title: {row['lesson_title']}",
        f"Lesson Summary: {row['lesson_summary']}",
        f"Part Title: {row['part_title']}",
        f"Part Summary: {row['part_summary']}",
        f"Program Title: {row['program_title']}",
        f"School: {row['primary_school']}",
        f"Level: {row['difficulty_level']}",
        f"Duration: {row['duration']}"
    ]
    return "\n".join([str(f) for f in fields if pd.notna(f)])

df['combined_text'] = df.apply(combine_lesson_text, axis=1)

df[['lesson_title', 'combined_text']].sample(5, random_state=7)

,lesson_title,combined_text
1727,Automation,Lesson Title: Automation\nLesson Summary: In t...
1536,Optimizing Minimax Search,Lesson Title: Optimizing Minimax Search\nLesso...
172,Introduction to Digital Project Management,Lesson Title: Introduction to Digital Project ...
3524,System Auditing,Lesson Title: System Auditing\nLesson Summary:...
1167,An Introduction to Your Nanodegree Program,Lesson Title: An Introduction to Your Nanodegr...


In [20]:
# ## Read saved embeddings
# df = pd.read_parquet("lesson_embeddings_openai.parquet")
# ## or 
# df = pd.read_csv("lesson_embeddings_openai.csv")

In [19]:
df[['lesson_title', 'embedding', 'openai_embedding']].sample(10, random_state=7)

,lesson_title,embedding,openai_embedding
1727,Automation,"[-0.012544128112494946, 0.0773787572979927, -0...","[-0.00270081777125597, -0.021109839901328087, ..."
1536,Optimizing Minimax Search,"[-0.048768047243356705, 0.033145494759082794, ...","[0.020191697403788567, -0.008061559870839119, ..."
172,Introduction to Digital Project Management,"[-0.014023364521563053, 0.022668685764074326, ...","[1.5199954759737011e-05, -0.017706571146845818..."
3524,System Auditing,"[-0.03462547808885574, 0.06654990464448929, -0...","[0.014628365635871887, -0.009944604709744453, ..."
1167,An Introduction to Your Nanodegree Program,"[-0.10374992340803146, -0.054070860147476196, ...","[0.014809690415859222, -0.006754512432962656, ..."
2711,"Building, Evaluating and Interpreting Models","[-0.031477466225624084, 0.015866808593273163, ...","[0.0014057692606002092, 0.010276060551404953, ..."
370,Control Flow,"[-0.04909806698560715, 0.02069883793592453, -0...","[0.0072598946280777454, -0.007735494524240494,..."
4830,Problem Set 1,"[-0.06380009651184082, -0.015353601425886154, ...","[0.01663575880229473, 0.0021367508452385664, 0..."
3766,"Big Data Ecosystem, Data Lakes, and Spark","[-0.003940301947295666, 0.006446739658713341, ...","[0.013580742292106152, -0.017822636291384697, ..."
2099,Turtles and Code,"[-0.03956529125571251, 0.06922376155853271, 0....","[0.023302091285586357, -0.012457811273634434, ..."


In [7]:
# ## embedding without openai
# from sentence_transformers import SentenceTransformer
# import numpy as np

# model = SentenceTransformer('all-MiniLM-L6-v2')

# # generate embeddings
# embeddings = model.encode(df['combined_text'].tolist(), show_progress_bar=True)

# df['embedding'] = embeddings.tolist()

/Users/huimin.yang/huimin_projects/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|████████████████████████████████| 156/156 [00:14<00:00, 10.42it/s]


In [8]:
## embedding with openai
from openai import OpenAI
import time

key = 'voc-1729574681126677149492966d214ffce5460.25792363'
client = OpenAI(
    base_url = "https://openai.vocareum.com/v1",
    api_key = key
)

In [15]:
# # initialize embedding column
# df['openai_embedding'] = None

# # parameters
# BATCH_SIZE = 20
# SAVE_EVERY = 10  # Save after every batch
# OUTPUT_FILE = "lesson_embeddings_openai.parquet"

# # loop over batches
# for i in range(0, len(df), BATCH_SIZE):
#     batch = df.iloc[i:i + BATCH_SIZE]
    
#     # skip rows already embedded
#     if batch['openai_embedding'].notnull().all():
#         continue

#     print(f"Embedding row {i} to {i + len(batch)}...")

#     try:
#         response = client.embeddings.create(
#             model="text-embedding-ada-002",
#             input=batch['combined_text'].tolist()
#         )
#         embeddings = [r.embedding for r in response.data]

#         for idx, emb in zip(batch.index, embeddings):
#             df.at[idx, 'openai_embedding'] = emb

#     except Exception as e:
#         print(f" Error in row {i}-{i + len(batch)}: {e}")
#         continue

#     # save progress
#     if i % SAVE_EVERY == 0:
#         df.to_parquet(OUTPUT_FILE, index=False)
#         print(f" Progress saved at row {i}")

#     time.sleep(0.2)

Embedding row 0 to 20...
 Progress saved at row 0
Embedding row 20 to 40...
 Progress saved at row 20
Embedding row 40 to 60...
 Progress saved at row 40
Embedding row 60 to 80...
 Progress saved at row 60
Embedding row 80 to 100...
 Progress saved at row 80
Embedding row 100 to 120...
 Progress saved at row 100
Embedding row 120 to 140...
 Progress saved at row 120
Embedding row 140 to 160...
 Progress saved at row 140
Embedding row 160 to 180...
 Progress saved at row 160
Embedding row 180 to 200...
 Progress saved at row 180
Embedding row 200 to 220...
 Progress saved at row 200
Embedding row 220 to 240...
 Progress saved at row 220
Embedding row 240 to 260...
 Progress saved at row 240
Embedding row 260 to 280...
 Progress saved at row 260
Embedding row 280 to 300...
 Progress saved at row 280
Embedding row 300 to 320...
 Progress saved at row 300
Embedding row 320 to 340...
 Progress saved at row 320
Embedding row 340 to 360...
 Progress saved at row 340
Embedding row 360 to 380..

In [74]:
def search_lessons(query, df, client, top_k=10):

    response = client.embeddings.create(
        model="text-embedding-ada-002",
        input=query
    )
    query_embedding = response.data[0].embedding

    lesson_embeddings = np.array(df['openai_embedding'].tolist())
    similarities = cosine_similarity([query_embedding], lesson_embeddings)[0]

    df_temp = df.copy()
    df_temp['similarity'] = similarities

    df_temp = df_temp.sort_values(by='similarity', ascending=False)

    # Drop duplicates by lesson title (keep the most relevant version)
    df_temp = df_temp.drop_duplicates(subset='lesson_title', keep='first')

    # Return top results
    return df_temp.head(top_k)

In [76]:
def display_recommendations(results, query):
    print(f"\n🔍 Recommendations for: '{query}'")
    for idx, row in results.iterrows():
        print("—" * 60)
        print(f" Lesson Title: {row['lesson_title']}")
        print(f" Lesson Summary: {row['lesson_summary'][:300] if pd.notna(row['lesson_summary']) else 'No summary available.'}")
        print(f" Program: {row['program_title']}")
        part_title = row['part_title'] if pd.notna(row['part_title']) else "No course title available."
        print(f" Course: {part_title}")
       # print(f"📈 Similarity: {row['similarity']:.3f}")

In [138]:
# intent classification
def classify_query_scope_gpt(query, client):
    prompt = f"""
You are an expert in learning content classification.

Given a user query, classify their learning intent into one of the following scopes:

- `lesson`: A very specific skill or technique, usually taught in a single lesson.  
  *Examples: "sql join", "python dictionaries", "design phase", "data cleaning", "for loops in C++"*

- `course`: A broader topic that includes multiple lessons, such as a course or unit.  
  *Examples: "practical statistics", "intro to python", "advanced data analysis", "sql fundamentals", "web development with Flask"*

- `nanodegree`: A full learning path or specialization made up of multiple courses, often covering an entire role or domain.  
  *Examples: "intermediate java", "generative AI", "data scientist", "android app developer", "machine learning engineer"*

---

### Example Queries:

| Query                          | Scope     |
|-------------------------------|-----------|
| "sql join"                    | lesson    |
| "polishing dashboards"        | lesson    |
| "python dictionaries"         | lesson    |
| "intro to python"             | course    |
| "web development with Flask"  | course   |
| "generative AI"               | nanodegree   |
| "data scientist"              | nanodegree  |
| "machine learning engineer"     | nanodegree  |

---

### Now classify this user query:

Query: "{query}"

Respond with one word only:  
`lesson`, `course`, or `nanodegree`

"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip().lower()

In [126]:
def smart_recommend(query, df, client, top_k=50):
    # step 1: detect scope using GPT
    scope = classify_query_scope_gpt(query, client)
    print(f"\n🔍 Detected scope: {scope.upper()} for query: '{query}'")

    # step 2: search all lessons
    results = search_lessons(query, df, client, top_k=top_k)

    # step 3: precompute deduplicated top results
    top_lessons = results.drop_duplicates(subset='lesson_title')
    top_parts = results.drop_duplicates(subset='part_title')
    top_programs = results.drop_duplicates(subset='program_title')

    # step 4: recommend based on detected scope
    if scope == "program":
        print(f"\n🏫 Recommended Programs:")
        for _, row in top_programs.head(2).iterrows():
            print("—" * 60)
            print(f"🏫 Program: {row['program_title']}")
            summary = row['part_summary'] if pd.notna(row['part_summary']) else "No summary available."
            print(f"📝 Sample Course summary: {summary[:300]}")

        print(f"\n📚 Recommended Courses:")
        for _, row in top_parts.head(3).iterrows():
            print("—" * 60)
            print(f"📚 Course: {row['part_title']}")
            summary = row['part_summary'] if pd.notna(row['part_summary']) else "No summary available."
            print(f"📝 Course summary: {summary[:300]}")
            print(f"🏫 Program: {row['program_title']}")

    elif scope == "part":
        print(f"\n📚 Recommended Courses:")
        for _, row in top_parts.head(3).iterrows():
            print("—" * 60)
            print(f"📚 Course: {row['part_title']}")
            summary = row['part_summary'] if pd.notna(row['part_summary']) else "No summary available."
            print(f"📝 Course summary: {summary[:300]}")
            print(f"🏫 Program: {row['program_title']}")

        print(f"\n🎯 Recommended Lessons:")
        for _, row in top_lessons.head(3).iterrows():
            print("—" * 60)
            print(f"🎯 Lesson: {row['lesson_title']}")
            summary = row['lesson_summary'] if pd.notna(row['lesson_summary']) else "No summary available."
            print(f"📝 Lesson ummary: {summary[:300]}")
            print(f"📚 Course: {row['part_title']}")

    elif scope == "lesson":
        print(f"\n🎯 Recommended Lessons:")
        for _, row in top_lessons.head(5).iterrows():
            print("—" * 60)
            print(f"🎯 Lesson: {row['lesson_title']}")
            summary = row['lesson_summary'] if pd.notna(row['lesson_summary']) else "No summary available."
            print(f"📝 Lesson summary: {summary[:300]}")
            print(f"📚 Course: {row['part_title']}")

    else:
        print(f"⚠️ Unrecognized scope '{scope}'. Showing lesson-level results by default.")
        for _, row in top_lessons.head(5).iterrows():
            print("—" * 60)
            print(f"🎯 Lesson: {row['lesson_title']}")
            summary = row['lesson_summary'] if pd.notna(row['lesson_summary']) else "No summary available."
            print(f"📝 Lesson summary: {summary[:300]}")
            print(f"📚 Course: {row['part_title']}")

In [146]:
query = "AWS Machine Learning Foundations"
classify_query_scope_gpt(query, client)

'course'

In [135]:
query = "SQL"
# 🚀 Run smart recommendation
smart_recommend(query, df, client)


🔍 Detected scope: PART for query: 'SQL'

📚 Recommended Courses:
————————————————————————————————————————————————————————————
📚 Course: SQL Fluency
📝 Course summary: SQL is one of the most versatile tools available for extracting insights from stored data. In this course, you will learn how databases work. You will also learn some basic SQL queries for that can be used to select, manipulate and aggregate data in databases.
🏫 Program: SQL Fluency
————————————————————————————————————————————————————————————
📚 Course: Introduction to SQL
📝 Course summary: SQL is one of the most versatile tools available for extracting insights from stored data.
Learn how to execute core SQL commands to define, select, manipulate, control access, aggregate, and join data and data tables. Understand when and how to use subqueries, several window functions, and partitio
🏫 Program: SQL
————————————————————————————————————————————————————————————
📚 Course: Intro to Relational Databases
📝 Course summary: No sum

In [79]:
query = "SQL"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'SQL'
————————————————————————————————————————————————————————————
 Lesson Title: Introduction to Relational Databases
 Lesson Summary: Learn the principles behind relational data organization: tables, queries, aggregations, keys, and joins.
 Program: SQL Fluency
 Course: SQL Fluency
————————————————————————————————————————————————————————————
 Lesson Title: Getting Started in SQL
 Lesson Summary: In this section, you will gain knowledge about SQL basics for working with a single table. You will learn the key commands to filter a table in many different ways.
 Program: SQL Fluency
 Course: SQL Fluency
————————————————————————————————————————————————————————————
 Lesson Title: SQL Joins
 Lesson Summary: In this lesson, you will learn how to combine data from multiple tables together.
 Program: SQL Fluency
 Course: SQL Fluency
————————————————————————————————————————————————————————————
 Lesson Title: Creating a Database Using Tables
 Lesson Summary: Create your o

In [133]:
query = "SQL"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'SQL'
————————————————————————————————————————————————————————————
 Lesson Title: Introduction to Relational Databases
 Lesson Summary: Learn the principles behind relational data organization: tables, queries, aggregations, keys, and joins.
 Program: SQL Fluency
 Course: SQL Fluency
————————————————————————————————————————————————————————————
 Lesson Title: Getting Started in SQL
 Lesson Summary: In this section, you will gain knowledge about SQL basics for working with a single table. You will learn the key commands to filter a table in many different ways.
 Program: SQL Fluency
 Course: SQL Fluency
————————————————————————————————————————————————————————————
 Lesson Title: SQL Joins
 Lesson Summary: In this lesson, you will learn how to combine data from multiple tables together.
 Program: SQL Fluency
 Course: SQL Fluency
————————————————————————————————————————————————————————————
 Lesson Title: Creating a Database Using Tables
 Lesson Summary: Create your o

In [66]:
query = "java"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'java'
————————————————————————————————————————————————————————————
 Lesson Title: Java Programing Basics
 Lesson Summary: This lesson covers all the major building blocks we need to create a simple Java app, including keywords, variables, loops, methods, access modifiers, arrays, and Javadoc.
 Program: Java Fundamentals
 Course: Java Fundamentals
————————————————————————————————————————————————————————————
 Lesson Title: Introduction to Java
 Lesson Summary: In this lesson, we'll cover some fundamentals—such as why you would want to learn Java, and what prerequisites you'll need to succeed in the course.
 Program: Java Fundamentals
 Course: Java Fundamentals
————————————————————————————————————————————————————————————
 Lesson Title: Introduction to Advanced Java Programming Techniques
 Lesson Summary: In this lesson, you'll be introduced to the course and what to expect!
 Program: Advanced Java Programming Techniques
 Course: Advanced Java Programming Technique

In [67]:
query = "machine learning"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'machine learning'
————————————————————————————————————————————————————————————
 Lesson Title: 5. Machine Learning
 Lesson Summary: No summary available.
 Program: Intro to Artificial Intelligence
 Course: Intro to Artificial Intelligence
————————————————————————————————————————————————————————————
 Lesson Title: The Machine Learning Workflow
 Lesson Summary: Machine learning is more than just building a model - getting each step of the workflow right is crucial.
 Program: Self Driving Car Engineer
 Course: Computer Vision
————————————————————————————————————————————————————————————
 Lesson Title: Automated ML and Hyperparameter Tuning
 Lesson Summary: We'll discuss how we can use the Azure ML SDK to do automated ML and create models more quickly and accurately. We'll also have a look at how we can make we can make our models easier to interpret.
 Program: Machine Learning Engineer with Microsoft Azure
 Course: Using Azure Machine Learning
——————————————————————

In [68]:
query = "free"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'free'
————————————————————————————————————————————————————————————
 Lesson Title: Dynamic Memory Allocation (The Heap)
 Lesson Summary: This lesson introduces dynamic memory allocation on the heap. The commands malloc and free as well as new and delete are introduced for this purpose.
 Program: C++
 Course: Memory Management
————————————————————————————————————————————————————————————
 Lesson Title: Monetization Models
 Lesson Summary: Discuss monetization strategies for both premium games and free-to-download “freemium” games.
 Program: Engagement & Monetization | Mobile Games
 Course: Engagement & Monetization | Mobile Games
————————————————————————————————————————————————————————————
 Lesson Title: Code Optimization
 Lesson Summary: This lesson equips you with the power to optimize code for performance, memory usage, and power efficiency in embedded systems, ensuring responsive and resource-efficient applications.
 Program: Embedded Systems Programming with 

In [70]:
query = "AI"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'AI'
————————————————————————————————————————————————————————————
 Lesson Title: AI/ML and AIoT
 Lesson Summary: In this lesson, we transition from IoT to AIoT , looking at intelligence and the role that it can play in IoT. We'll look at intelligence, along with basic techniques and concerns of AIoT.
 Program: Embedded Systems, IoT, and AIoT
 Course: Embedded Systems, IoT, and AIoT
————————————————————————————————————————————————————————————
 Lesson Title: AI for Accessibility
 Lesson Summary: Integrate accessibility and AI. Learn about key technologies and apply best practices for inclusive design and development throughout AI projects
 Program: AI for Accessibility
 Course: AI for Accessibility
————————————————————————————————————————————————————————————
 Lesson Title: Business Strategy
 Lesson Summary: In this lesson, we'll learn how to apply the AIoT framework to business strategy, and we'll consider three types of AIoT use cases (Digital OEM, Digital Equipm

In [71]:
query = "C++"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'C++'
————————————————————————————————————————————————————————————
 Lesson Title: Introduction to C++
 Lesson Summary: C++ is one of the most important programming languages in the world. It is particularly important for high-performance software.
 Program: C++
 Course: Welcome
————————————————————————————————————————————————————————————
 Lesson Title: Introduction to the C++ Language
 Lesson Summary: In this lesson, you will build on your previous programming experience to learn the basics of the C++ language.
 Program: C++
 Course: C++ Foundations
————————————————————————————————————————————————————————————
 Lesson Title: Practical C++
 Lesson Summary: Learn how to write C++ code on your own computer and compile it into a executable program without running into too many compilation errors.
 Program: Computer Vision
 Course: C++ Programming
————————————————————————————————————————————————————————————
 Lesson Title: Course Outro
 Lesson Summary: No summary avail

In [72]:
query = "android"
results = search_lessons(query, df, client, top_k=5)
display_recommendations(results, query)


🔍 Recommendations for: 'android'
————————————————————————————————————————————————————————————
 Lesson Title: Design for Android Amazement
 Lesson Summary: No summary available.
 Program: UX Design for Mobile Developers
 Course: UX Design for Mobile Developers
————————————————————————————————————————————————————————————
 Lesson Title: Design and Build an Android Application
 Lesson Summary: In this project, you will have the opportunity to design and build either a custom Android application inspired by your own idea or an example provided to you.
 Program: Android Kotlin Developer
 Course: Android Capstone Project
————————————————————————————————————————————————————————————
 Lesson Title: Designing for Everyone
 Lesson Summary: Coming up with a good design for an app is always hard, but creating a great design is more than just colors and layouts. Let's build an app that is designed for everyone, everywhere!
 Program: Developing Android Apps with Kotlin
 Course: Developing Android App

In [ ]:

# Title text.
st.header('Prototype: Lesson Search')
st.write('A prototype of lesson discovery search engine that uses lesson title, lesson summary, course title, course summary, program title, and program summary to index results. It is biased toward matching key words against lesson metadata first, then course, then program.')
st.markdown(f'##### Lessons indexed: {len(df)}')

# Split two columns.
col1, col2 = st.columns(2)

# Text input for search.
with col1:
	text_search = st.text_input(f"Search for lessons by keyword search", value="")

# Filter to simulate filtering content from Workera Assessment.
with col2:
	workera_assessment_score = st.radio(f"Workera Assessment results", ["Didn't take an assessment","Beginner (0-100)", "Developing (101-200)","Accomplished (201-300)"])


"""
Series of functions to lazily search metadata based on input terms.
"""

def count_matches(text, words):
    """Counts the number of search words appearing in a given text."""
    if pd.isna(text):
        return 0
    return sum(word in text.lower() for word in words)

def filter_dataframe(df, text_search):
    """Filters the DataFrame based on search input and assigns ranking scores."""
    if not text_search:
        return df.copy()
    
    search_words = text_search.lower().split()
    match_columns = {
        'program_match_count': 'program_title',
        'course_title_match_count': 'part_title',
        'course_summary_match_count': 'part_summary',
        'lesson_title_match_count': 'lesson_title',
        'lesson_summary_match_count': 'lesson_summary',
    }
    
    for col, source in match_columns.items():
        df[col] = df[source].apply(lambda x: count_matches(x, search_words))
    
    df_search = df[df[list(match_columns.keys())].sum(axis=1) > 0]
    
    # Higher weighting for lesson title and lesson summary.
    df_search['rank_score'] = (
        df_search['lesson_title_match_count'] * 5 +
        df_search['lesson_summary_match_count'] * 4 +
        df_search['course_title_match_count'] * 3 +
        df_search['course_summary_match_count'] * 2 +
        df_search['program_match_count'] * 1
    )
    
    df_search['match_count'] = df_search[list(match_columns.keys())].sum(axis=1)
    
    return df_search.sort_values(by=['rank_score', 'match_count'], ascending=[False, False])

def apply_difficulty_filter(df_search, workera_assessment_score):
    """Filters the DataFrame based on difficulty level."""
    difficulty_mapping = {
        "Beginner (0-100)": ['Fluency', 'Beginner', 'Discovery'],
        "Developing (101-200)": ['Intermediate'],
        "Accomplished (201-300)": ['Advanced']
    }
    
    if workera_assessment_score in difficulty_mapping:
        df_search = df_search[df_search['difficulty_level'].isin(difficulty_mapping[workera_assessment_score])]
    
    return df_search

def display_results(df_search):
    """Displays search results as cards in Streamlit."""
    st.markdown(f"##### {len(df_search)} lessons found")
    
    N_cards_per_row = 1
    for n_row, row in df_search.reset_index().iterrows():
        i = n_row % N_cards_per_row
        if i == 0:
            st.write("---")
            cols = st.columns(N_cards_per_row, gap="large")
        with cols[i]:
            st.subheader(row['lesson_title'])
            st.markdown(row['lesson_summary'])
            st.markdown(f"_Program: {row['program_title']} > Course: {row['part_title']} > Lesson: {row['lesson_title']}_")

# Streamlit App Logic
if st.button('search'):
    df_search = filter_dataframe(df, text_search)
    df_search = apply_difficulty_filter(df_search, workera_assessment_score)
    display_results(df_search)